# Evaluation of performance of the algorithm on Yale Database 

### Imports & Définition de fonctions utiles

In [1]:
import random
random.seed(2017)
import time
import os, sys

In [4]:
from algorithme1 import *
import algorithme1

In [7]:
def read_images(path, sz=None,sz0=168,sz1=192): 
    '''
    Chargement des données
    Si spécifiée, réduction de dimensions incluse
    '''
    c=0
    X,y = [], []
    for dirname , dirnames , filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname , subdirname) 
            for filename in os.listdir(subject_path):
                try:
                    im = Image.open(os.path.join(subject_path , filename)) 
                    im = im.convert("L")
                    # resize to given size (if given) and check that it's the good size
                    if ((im.size[0] == sz0) & (im.size[1]==sz1)):
                        if (sz is not None):
                            im = im.resize(sz, Image.NEAREST)     
                        X.append(np.asarray(im, dtype=np.uint8)) 
                        y.append(c)
                except IOError:
                    pass
                except:
                    print ("Unexpected error:", sys.exc_info()[0] )
                    raise
            c = c+1
            
    print ("Images chargees !")
    return [X,y]

### Chargement des images

In [8]:
path_images = 'data/CroppedYale/' # On détermine ici sur quelles images on veut s'entrainer

# D'abord on charge les images originales (utilisées pour la visualisation et la reconstruction)

X_original, y_original = read_images(path_images, sz=None)

Images chargees !


### Separation in training set and test set

In [9]:
Xtrain, Xtest = [], []
ytrain, ytest = [], []
indices_train, indices_test = [], []

for i in range(len(X_original)):
    if i%64==0:
        # On compute les indices des éléments à placer dans le test, ils sont différents pour chaque image.
        test1 = np.random.choice(28,5,replace=False)
        test2 = 29 + np.random.choice(35,7,replace=False)
    
    # On crée X_test et X_train
    if ((i%64 in test1) or (i%64 in test2)):
        Xtest.append(X_original[i])
        ytest.append(y_original[i])
        indices_test.append(i)
    else:
        Xtrain.append(X_original[i])
        ytrain.append(y_original[i])
        indices_train.append(i)

## Obtention du meilleur lambda

Note : la méthode d'obtention n'est pas très rigoureuse mais il est içi relativement difficile de faire des train / test sets équilibrés sachant que certaines images sont vraiment trop noires, et qu'une cross validation à la main prendrait un peu de temps.

In [13]:
alpha_vec = [0.002]
methods = [None,'fisherfaces']
noise=[0,0.1,0.3,0.4,0.5]
accuracy = np.zeros((len(methods),len(alpha_vec)))

for i, alpha in enumerate(alpha_vec):
    for j,m in enumerate(methods):
        # Génération des résultats
        L = algorithme1.SRC(Xtrain, Xtest, ytrain, type_feature_reduc=m, reduce_lines=12, reduce_columns=10, lambda_val=alpha, per_bruit=0.1, pos_occl=None)
        
        # Obtention de la précision
        predictions = []
        for a,b,c in L:
            predictions.append(b)
        predictions = np.asarray(predictions)
        accuracy[j,i] = 100.0*np.sum((predictions == ytest))/len(Xtest)
    print("lambda = ", alpha, ", fini")
        
        

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause preci

lambda =  0.002 , fini


In [14]:
accuracy

array([[ 71.27192982]])

In [ ]:
plt.figure()
plt.clf()
for j,m in enumerate(methods):
    plt.plot(accuracy[:,j])
    plt.title('Reduction de dimension : ', m)